In [ ]:
%pip install -q git+https://github.com/CompVis/taming-transformers.git
!git clone https://github.com/CompVis/taming-transformers.git && cd taming-transformers && python -m pip install -e .
%pip install "omegaconf>=2.0.0" "pytorch-lightning>=1.0.8" einops transformers imageio-ffmpeg
%pip install -q git+https://github.com/cthiounn/dalle-tiny.git

!curl -L "https://huggingface.co/boris/vqgan_f16_16384/raw/main/config.yaml" > "config_vqgan_minidalle.yaml"
!curl -L "https://huggingface.co/boris/vqgan_f16_16384/resolve/main/model.ckpt" > "model_vqgan_minidalle.ckpt"

import os
os._exit(00)

In [ ]:
import sys
sys.path.append(".")

# also disable grad to save memory
from omegaconf import OmegaConf
import taming
from taming.models.vqgan import VQModel
import torch
torch.set_grad_enabled(False)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



config_path = "config_vqgan_minidalle.yaml"
config = OmegaConf.load(config_path)
vqmodel=VQModel(**config.model.params).to(DEVICE)

ckpt_path = "model_vqgan_minidalle.ckpt"

sd = torch.load(ckpt_path, map_location=DEVICE)["state_dict"]
vqmodel.load_state_dict(sd, strict=False)

In [ ]:
from dalle_tiny.model import TinyDalleModel
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

device = 'cuda' if torch.cuda.is_available() else 'cpu'

from transformers import BartForConditionalGeneration

model=TinyDalleModel.from_pretrained('facebook/bart-large-cnn')
model.reinit_lm_head_for_images()
model.final_logits_bias=torch.rand(16384)
model.final_logits_bias[:]=0


In [ ]:
from transformers import BartTokenizer
caption="a field of grass and sunflowers"
tokenizer=BartTokenizer.from_pretrained('facebook/bart-large-cnn')
inputs=tokenizer(caption, return_tensors="pt",max_length=255,padding="max_length")
inputs=inputs.to(device)

In [ ]:
import matplotlib.pyplot as plt


model.load_state_dict(torch.load('checkpoint_115.pth',map_location=device))
model.eval()
model=model.to(device)
pred=model.generate(**inputs, do_sample=True, top_k=100)
print(pred)
pred=pred.detach()
pred=pred.squeeze()
output_indices=torch.Tensor(256)
output_indices[:]=0
output_indices[:141]=pred[1:]
output_indices=output_indices.to(torch.int)
output_indices.to("cpu")
vqmodel=vqmodel.to("cpu")


z_q = vqmodel.quantize.embedding(output_indices).reshape(1, 16, 16, 256).permute(0,3,1,2)
u=vqmodel.decode(z_q).add(1).div(2).cpu().squeeze().permute(1, 2, 0)
plt.imshow(u)